## Tutorial: Create an Isolated VLAN and Servers

This notebook demonstrates how to create isolated VLANs and attach servers.

## Tutorial: 

#### Configure the Environment

In [ ]:
import json
import os
import chi
from datetime import datetime, timedelta
from dateutil import tz

from chi.server import *
from chi.lease import *
from chi.network import *

#Config with your project and site
chi.use_site('CHI@UC')
chi.set('project_name', 'CH-822154')

#Insert keypair name
key_name = 'my_chameleon_key'  # Change to your keypair

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
server_name = username+'Server'
network_name = username+'Net'
subnet_name = username+'Subnet'
router_name = username+'Router'
lease_name = username+'Lease2'

#Server attributes
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_haswell"
server_count=1
physical_network="physnet1"
BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'
DEFAULT_CIDR = '192.168.1.0/24'

#### Create a Lease

In [ ]:
# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

In [ ]:
#Get the lease by name
lease_id = get_lease_id(lease_name)
lease = get_lease(lease_id)
    
#Print the lease info
print(json.dumps(lease, indent=2))

#### Get the Reservations

In [ ]:
compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', lease['reservations']))[0]['id']
floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', lease['reservations']))[0]['id']

print("compute_reservation_id: " + compute_reservation_id)
print("floatingip_reservation_id: " + floatingip_reservation_id)

#### Create the Network

In [ ]:
# Create a network out of provider network physnet1
create_network(network_name, provider='physnet1')

In [ ]:
network = get_network(network_name)
network_id = network['id']

print('Network ID: ' + str(network_id))

#### Configure the Network

In [ ]:
subnet = create_subnet(subnet_name, network_id, cidr=DEFAULT_CIDR, gateway_ip=None)
router = create_router(router_name, gw_network_name='public')
add_subnet_to_router(router['id'], subnet['id'])

#### Start the Server

In [ ]:
#create the server
server = create_server(server_name, 
                       compute_reservation_id, 
                       key_name=key_name, 
                       network_id=network_id, 
                       network_name=network_name, 
                       nics=[], 
                       image_id=None, 
                       image_name=image_name, 
                       flavor_id=None, 
                       flavor_name=flavor_name, 
                       count=1)
server_id = get_server_id(server_name)

#### Associate the Floating IP   (TODO: need to find floating_ip from the reservation that was just made)

In [ ]:
floating_ip = associate_floating_ip(server_id)

print('Floating IP: ' + str(floating_ip))

#### Delete Server

In [ ]:
delete_server(server_id)

#### De-configure Network

In [ ]:
try:
    result = remove_subnet_from_router(router['id'], subnet['id'])
except Exception as e:
    print("remove_subnet_from_router error: " + str(e))
    pass

try:
    result = delete_router(router['id'])
except Exception as e:
    print("delete_router error: " + str(e))
    pass

try:
    result = delete_subnet(subnet['id'])
except Exception as e:
    print("delete_subnet error: " + str(e))
    pass

try:
    result = delete_network(network_id)
except Exception as e:
    print("delete_network error: " + str(e))
    pass

#### Release Lease

In [ ]:
delete_lease(lease_id)